# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data and Orders API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module. It shows you how to filter for clouds based on a specific AOI.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)
* [rasterio](https://rasterio.readthedocs.io/en/latest/index.html)
* [shapely](https://shapely.readthedocs.io/en/stable/index.html)
* [asyncio](https://docs.python.org/3/library/asyncio.html)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Planet Client V2 Documentation](https://github.com/planetlabs/planet-client-python)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API and Orders API

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [ ]:
#general packages
import os
import json
import asyncio
import rasterio
import numpy as np
import nest_asyncio
from datetime import datetime

#geospatial packages
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import shape
from shapely.ops import unary_union
from shapely.geometry import mapping

#planet SDK
from planet import Auth
from planet import Session, data_filter


# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PASTE_API_KEY_HERE'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

Let's also read in a GeoJSON geometry into a variable so we can use it during testing. The geometry can only have one polygon to work with the data API

In [ ]:
with open("sf_all.geojson") as f:
    geom_all = json.loads(f.read())


### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API. Additional filters are described [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/#filter:~:text=(main())-,Filter,-%C2%B6).

In [ ]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

#Geometry filter
geom_filter = data_filter.geometry_filter(geom_all)

#Date range filter
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=5, day=1, year=2023))#, lt = datetime(month=2, day=1, year=2023))

#Cloud cover filter
clear_percent_filter = data_filter.range_filter('clear_percent', 0,None)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter])

In [ ]:
combined_filter

Now let's build the request:

In [ ]:
async with Session() as sess:
    cl = sess.client('data')
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

In [ ]:
request

In [ ]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our result limit to 50.
async with Session() as sess:
    cl = sess.client('data')
    items = cl.run_search(search_id=request['id'], limit=1000)
    item_list = [i async for i in items]

In [ ]:
print(len(item_list))

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

Then we can save the output to be visualized as a geojson

Now, we can iterate through our search results.

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['clear_percent'])

In [ ]:
geoms = {
  "type": "FeatureCollection",
  "features": []
}

if not os.path.isdir('output'):
    os.mkdir('output')
else:
    if os.path.isfile('output/results.geojson'):
        os.remove('output/results.geojson')

with open('output/results.geojson','w') as f:
    for item in item_list:
        geom_out =     {
          "type": "Feature",
          "properties": {},
          "geometry": item['geometry']
        }
        geoms['features'].append(geom_out)
    jsonStr = json.dumps(geoms)
    f.write(jsonStr)
    f.close()

Now we can import our multiple geometries

In [ ]:
with open("sf_84.geojson") as f:
    geom_84 = json.loads(f.read())
geom_84 = geom_84['features']

A function that takes the geometry of the scenes and compares them with the AOIs in order to measuer coverage

In [ ]:
#Calculate the area of overlap between two geometries
def get_overlap(geometry1, geometry2):
    # Parse the JSON into geometry objects.
    shape1 = unary_union([shape(geom_1['geometry']) for geom_1 in geometry1])
    shape2 = shape(geometry2)

    # Compute the intersection of the two geometries.
    intersection = shape1.intersection(shape2)

    # Compute the areas of the geometries and their intersection.
    area1 = shape1.area
    area2 = shape2.area
    intersection_area = intersection.area

    # Compute the overlap as a percentage of the total area.
    if intersection_area == 0:
        return 0
    overlap = intersection_area / area1 * 100

    return overlap

Use the filter function in order to get 100% coverage over your two AOIs

In [ ]:
#recreate a geometry
geoms = {
  "type": "FeatureCollection",
  "features": []
}

#make a new list of IDs
covered_list = []


with open('output/results_coverage.geojson','w') as f:
    for item in item_list:
        overlap = get_overlap(geom_84, item['geometry'])
        print(overlap)
        if overlap >= 100:
            scene =     {
              "type": "Feature",
              "properties": {},
              "geometry": item['geometry']
            }
            geoms['features'].append(scene)
            covered_list.append(item)
    jsonStr = json.dumps(geoms)
    f.write(jsonStr)
    f.close()

Comparing the number of items overall compared to the ones that cover the entirety of our two AOIs

In [ ]:
print(len(item_list))
print(len(covered_list))

This GeoJSON file can be opened and viewed in any compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads. Let's start by looking at one item and the assets available to download for that item.

For more information on Items and Assets, check out [Items & Assets](https://developers.planet.com/docs/apis/data/items-assets/) on the Planet Developer Center.

In [ ]:
# As an example, let's look at the first result in our item_list and grab the item_id and item_type:
item = covered_list[0]
print(indent(item))
print(item['id'], item['properties']['cloud_percent'])

There are a few steps involved in order to download an asset using the Planet Python Client:

* **Get Asset:** Get a description of our asset based on the specifications we're looking for
* **Activate Asset:** Activate the asset with the given description
* **Wait Asset:** Wait for the asset to be activated
* **Download Asset:** Now our asset is ready for download!

Let's go through these steps below. We'll do this for our cloud asset the ortho_udm2.

In [ ]:
nest_asyncio.apply()

async def download_cloud(item):
    async with Session() as sess:
        cl = sess.client('data')
        # Get Asset
        asset_desc = await cl.get_asset(item_type_id=item['properties']['item_type'],item_id=item['id'], asset_type_id='ortho_udm2')
        # Activate Asset
        await cl.activate_asset(asset=asset_desc)
        # Wait Asset
        await cl.wait_asset(asset=asset_desc)
        # Download Asset
        for i in range(5):  # retry 3 times
            try:
                asset_path = await cl.download_asset(asset=asset_desc, directory='cloud_output', overwrite=True)
                return asset_path
            except Exception as e:
                print(f"Attempt {i+1} failed with error: {e}")
                if i < 4:  # if not the last attempt
                    await asyncio.sleep(5)  # wait for 5 seconds before retrying
                else:
                    raise  # re-raise the last exception if all attempts fail


        asset_path = await cl.download_asset(asset=asset_desc, directory='cloud_output', overwrite=True)
        return asset_path

In [ ]:
await download_cloud(item)

A function that takes in a geometry and a cloud masks and outputs the percent of clear imagery within the AOI

In [ ]:
def calculate_mask_coverage(file_path, geometry):
    # Convert geometry to GeoDataFrame
    gdf = gpd.GeoDataFrame([1], geometry=[geometry], crs='32610')

    # Open the geotiff file
    with rasterio.open(file_path) as src:
        # Transform geometry to raster CRS
        gdf = gdf.to_crs(src.crs)

        # Mask the geotiff with the geometry
        out_image, out_transform = mask(src, [mapping(gdf.geometry.values[0])], crop=True, filled=False)

        # Band 1 is the mask layer
        mask_band = out_image[0]

        # Convert the masked array to a regular numpy array and set a specific value for the masked pixels
        mask_band = np.where(mask_band.mask, -1, mask_band)
        
        # Calculate the total number of pixels
        total_pixels = np.sum(mask_band >= 0)  # Only count pixels with value 0 or 1

        # Calculate the number of 1s (True) and 0s (False)
        unique, counts = np.unique(mask_band[mask_band >= 0], return_counts=True)
        counts_dict = dict(zip(unique, counts))

        # Calculate the percentages
        percent_ones = (counts_dict.get(1, 0) / total_pixels) * 100
        percent_zeros = (counts_dict.get(0, 0) / total_pixels) * 100

        return percent_ones


We need a geojson that has the same CRS as the images which in this case is UDM 10N

In [ ]:
with open("sf_UTM.geojson") as f:
    geom_utm = json.loads(f.read())
geom_utm = geom_utm['features']

In [ ]:
calculate_mask_coverage("cloud_output/20240616_190907_10_2477_3B_udm2.tif",shape(geom_utm[0]['geometry']))

Now we run the function to download all the UDMs asyncronously. The output is rather busy so it is being stored in the captured variable

In [ ]:
%%capture captured

nest_asyncio.apply()

async with Session() as sess:
    tasks = [download_cloud(item) for item in covered_list]
    await asyncio.gather(*tasks)

    
print(captured.stderr)

This reads all of the cloud tiff files and creates a list of cloud cover over each AOI as well as a dictionary for each with scene ID and cloud values

In [ ]:
import glob

cloud_tifs = glob.glob("cloud_output/*")
sunset = []
mission = []
sunset_free = {}
mission_free = {}
for cloud in cloud_tifs:
    sunset_free[cloud] = calculate_mask_coverage(cloud,shape(geom_utm[1]['geometry']))
    mission_free[cloud] = calculate_mask_coverage(cloud,shape(geom_utm[0]['geometry']))
    sunset.append(sunset_free[cloud])
    mission.append(mission_free[cloud])
    

### Cloud Percent

Now you print out the cloud cover of each AOI

In [ ]:
print(np.mean(sunset))
print(np.mean(mission))

## Ordering

In this example, we will order a PSScene ortho_visual image. For variations on this kind of order, see Ordering Data.

In this order, we request a visual bundle. A bundle is a group of assets for an item. See the Scenes Product Bundles Reference to learn about other bundles and other items.

### Place Order
Create the order structure using `planet` functions

In [ ]:
from planet import order_request


async def assemble_order(item_ids):
    products = [
        order_request.product(item_ids, 'visual', 'PSScene')
    ]

    tools = [order_request.clip_tool(aoi=geom_all)]

    request = order_request.build_request(
        'test_order_sdk_method_2', products=products, tools=tools)
    return request
    
request =  await assemble_order(['20240615_190730_26_24d7'])

Having created the order we can now place it and await it for download

In [ ]:
from planet import reporting, Session, OrdersClient


# an async Orders client to request order creation
async with Session() as sess:
    cl = OrdersClient(sess)
    with reporting.StateBar(state='creating') as bar:
        # create order via Orders client
        order = await cl.create_order(request)
        bar.update(state='created', order_id=order['id'])

        # poll...poll...poll...
        await cl.wait(order['id'], callback=bar.update_state)

    # if we get here that means the order completed. Yay! Download the files.
    await cl.download_order(order['id'])

Now lets put it in as a function

In [ ]:
async def do_order(order):
    async with Session() as sess:
        cl = OrdersClient(sess)
        with reporting.StateBar(state='creating') as bar:
            order = await cl.create_order(order)
            bar.update(state='created', order_id=order['id'])

            await cl.wait(order['id'], callback=bar.update_state)

        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'])


Now we can order all the scenes at once

In [ ]:
ids = []
for info in covered_list:
    ids.append(info['id'])


request = await assemble_order(ids)
print(request)
await do_order(request)

In [ ]:
nest_asyncio.apply()


#now all you need to do to have them run in parallel is to create an array of order requests
async with Session() as sess:
    tasks = [do_order(o) for o in order_list]
    await asyncio.gather(*tasks)